# Block 1 — Imports

In [ ]:


import os
import numpy as np
import pandas as pd

from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
)

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX


# Block 2 — Paths and load CSVs

In [ ]:


BASE_PATH   = "./dataset/"
OUTPUT_PATH = "./outputs/"
os.makedirs(OUTPUT_PATH, exist_ok=True)

TRAIN_PATH  = os.path.join(BASE_PATH, "train.csv")
LABELS_PATH = os.path.join(BASE_PATH, "train_labels.csv")
TEST_PATH   = os.path.join(BASE_PATH, "test.csv")
PAIRS_PATH  = os.path.join(BASE_PATH, "target_pairs.csv")

train        = pd.read_csv(TRAIN_PATH)
train_labels = pd.read_csv(LABELS_PATH)
test         = pd.read_csv(TEST_PATH)
target_pairs = pd.read_csv(PAIRS_PATH)

target_cols = [c for c in train_labels.columns if c.startswith("target_")]

print("train :", train.shape)
print("labels:", train_labels.shape)
print("test  :", test.shape)
print("targets:", len(target_cols), " first 5:", target_cols[:5])

display(train.head(2))
display(train_labels[target_cols].head(2))
display(target_pairs.head(2))


train : (1961, 558)
labels: (1961, 425)
test  : (134, 559)
targets: 424  first 5: ['target_0', 'target_1', 'target_2', 'target_3', 'target_4']


,date_id,LME_AH_Close,LME_CA_Close,LME_PB_Close,LME_ZS_Close,JPX_Gold_Mini_Futures_Open,JPX_Gold_Rolling-Spot_Futures_Open,JPX_Gold_Standard_Futures_Open,JPX_Platinum_Mini_Futures_Open,JPX_Platinum_Standard_Futures_Open,...,FX_GBPCAD,FX_CADCHF,FX_NZDCAD,FX_NZDCHF,FX_ZAREUR,FX_NOKGBP,FX_NOKCHF,FX_ZARCHF,FX_NOKJPY,FX_ZARGBP
0,0,2264.5,7205.0,2570.0,3349.0,NaN,NaN,NaN,NaN,NaN,...,1.699987,0.776874,0.888115,0.689954,0.066653,0.090582,0.11963,0.078135,13.822740,0.059163
1,1,2228.0,7147.0,2579.0,3327.0,NaN,NaN,NaN,NaN,NaN,...,1.695279,0.778682,0.889488,0.692628,0.067354,0.091297,0.12052,0.079066,13.888146,0.059895


,target_0,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,...,target_414,target_415,target_416,target_417,target_418,target_419,target_420,target_421,target_422,target_423
0,0.005948,-0.002851,-0.004675,-0.000639,NaN,NaN,-0.006729,0.006066,NaN,0.003446,...,NaN,0.021239,-0.005595,NaN,-0.004628,0.033793,NaN,0.038234,NaN,0.02731
1,0.005783,-0.024118,-0.007052,-0.018955,-0.031852,-0.019452,0.003002,-0.006876,-0.002042,0.021284,...,0.003377,0.021372,-0.001517,0.012846,0.010547,0.030527,-0.000764,0.025021,0.003548,0.02094


,target,lag,pair
0,target_0,1,US_Stock_VT_adj_close
1,target_1,1,LME_PB_Close - US_Stock_VT_adj_close


# Block 3 — Helper: prune bad/constant features

In [ ]:


def prune_features(df: pd.DataFrame, var_thresh: float = 1e-12) -> pd.DataFrame:
    """
    Replace inf/NaN with 0 and drop columns with (almost) zero variance.
    """
    df = df.replace([np.inf, -np.inf], np.nan).fillna(0.0)
    keep = df.var(axis=0) > var_thresh
    return df.loc[:, keep]


# Block 4 — Baseline feature matrices (Model 1)

In [ ]:


X_base_full      = train.copy()
X_test_base_full = test.copy()

Xy_base = X_base_full.join(train_labels[target_cols]).dropna()

y_base = Xy_base[target_cols]
X_base = Xy_base.drop(columns=target_cols)

X_test_base_full = X_test_base_full.fillna(0.0)

print("Baseline merged shapes:")
print("X_base      :", X_base.shape)
print("y_base      :", y_base.shape)
print("X_test_base :", X_test_base_full.shape)

X_base_pruned = prune_features(X_base)
X_test_base_pruned = X_test_base_full.reindex(columns=X_base_pruned.columns, fill_value=0.0)

print("Baseline feature count:", X_base.shape[1], "→ after prune:", X_base_pruned.shape[1])


Baseline merged shapes:
X_base      : (144, 558)
y_base      : (144, 424)
X_test_base : (134, 559)
Baseline feature count: 558 → after prune: 558


# Block 5 — TimeSeries CV for Baseline LGBM (Model 1)


In [ ]:


base_est = LGBMRegressor(
    n_estimators=800,
    learning_rate=0.05,
    num_leaves=64,
    max_depth=-1,
    min_data_in_leaf=20,
    feature_fraction=0.9,
    bagging_fraction=0.8,
    bagging_freq=1,
    random_state=42,
    n_jobs=-1,
    verbosity=-1,
)

base_model = MultiOutputRegressor(base_est)
tscv_base = TimeSeriesSplit(n_splits=5)

baseline_metrics = []

print("===== [MODEL 1] Baseline LGBM — TimeSeries CV =====")
for fold, (tr_idx, va_idx) in enumerate(tscv_base.split(X_base_pruned), 1):
    X_tr, X_va = X_base_pruned.iloc[tr_idx], X_base_pruned.iloc[va_idx]
    y_tr, y_va = y_base.iloc[tr_idx], y_base.iloc[va_idx]

    base_model.fit(X_tr, y_tr)
    preds = base_model.predict(X_va)

    y_true = y_va.to_numpy().ravel()
    y_pred = preds.ravel()

    mse  = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae  = mean_absolute_error(y_true, y_pred)
    r2   = r2_score(y_true, y_pred)

    baseline_metrics.append([fold, rmse, mae, r2])
    print(f"[BASELINE] Fold {fold} → RMSE={rmse:.6f}, MAE={mae:.6f}, R2={r2:.4f}")

baseline_df = pd.DataFrame(baseline_metrics, columns=["Fold","RMSE","MAE","R2"])
display(baseline_df)
print("\nBaseline means:\n", baseline_df.mean(numeric_only=True))


===== [MODEL 1] Baseline LGBM — TimeSeries CV =====
[BASELINE] Fold 1 → RMSE=0.030059, MAE=0.020603, R2=-0.0964
[BASELINE] Fold 2 → RMSE=0.025478, MAE=0.018874, R2=-0.2616
[BASELINE] Fold 3 → RMSE=0.028952, MAE=0.020771, R2=-0.3183
[BASELINE] Fold 4 → RMSE=0.035986, MAE=0.026550, R2=-0.6331
[BASELINE] Fold 5 → RMSE=0.032946, MAE=0.023918, R2=-0.2941


,Fold,RMSE,MAE,R2
0,1,0.030059,0.020603,-0.096402
1,2,0.025478,0.018874,-0.261552
2,3,0.028952,0.020771,-0.318257
3,4,0.035986,0.026550,-0.633122
4,5,0.032946,0.023918,-0.294107



Baseline means:
 Fold    3.000000
RMSE    0.030684
MAE     0.022143
R2     -0.320688
dtype: float64


# Block 6 — Train Baseline LGBM on full data and save submission

In [ ]:


base_model.fit(X_base_pruned, y_base)
preds_test_base = base_model.predict(X_test_base_pruned)

sub_base = pd.DataFrame(preds_test_base, columns=target_cols)
os.makedirs(OUTPUT_PATH, exist_ok=True)
baseline_sub_path = os.path.join(OUTPUT_PATH, "submission_baseline_full_lgbm.csv")
sub_base.to_csv(baseline_sub_path, index=False)

print(f"Saved BASELINE submission to: {baseline_sub_path}")
display(sub_base.head())


Saved BASELINE submission to: ./outputs/submission_baseline_full_lgbm.csv


,target_0,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,...,target_414,target_415,target_416,target_417,target_418,target_419,target_420,target_421,target_422,target_423
0,-0.003510,0.013383,-0.013211,-0.005996,0.003198,0.018227,0.009919,0.003332,2.350666e-03,-0.002377,...,0.003752,-0.014817,0.003858,0.008874,0.002023,-0.001390,0.017864,-0.022761,-0.006488,-0.045803
1,-0.000952,0.021933,0.007766,0.013416,-0.017804,-0.015222,0.016672,-0.006810,-1.426140e-07,-0.013912,...,0.012316,-0.019526,0.017454,0.009694,0.027980,-0.031389,0.019280,-0.034649,0.003845,-0.076583
2,0.002198,0.011809,-0.003734,-0.001130,0.001957,0.007023,0.009940,-0.003242,3.250249e-03,0.003593,...,0.005913,-0.010343,0.007221,0.004549,0.000561,-0.006468,0.018959,-0.033860,-0.006195,-0.026995
3,-0.001527,0.009089,-0.006566,0.001078,0.004742,0.012349,0.008885,0.001842,-1.693807e-03,0.001828,...,0.004315,-0.008812,0.002841,0.003554,0.008072,-0.010797,0.024614,-0.032323,-0.008263,-0.026019
4,-0.007459,0.013754,-0.008964,0.001355,0.005242,0.016080,0.011826,0.001698,-5.415257e-03,-0.003371,...,0.007187,-0.009384,0.003289,0.004229,0.016640,-0.002805,0.023771,-0.026084,-0.006937,-0.016782


# Block 7 — Domain feature builders (Model 2)

In [ ]:


from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def split_domains(df: pd.DataFrame) -> dict:
    groups = {
        "US_Stock": [c for c in df.columns if c.startswith("US_Stock_")],
        "LME":      [c for c in df.columns if c.startswith("LME_")],
        "JPX":      [c for c in df.columns if c.startswith("JPX_")],
        "FX":       [c for c in df.columns if c.startswith("FX_")],
    }
    return {k: v for k, v in groups.items() if len(v) > 0}


def make_domain_features(df, name, cols, n_pca=3, roll_windows=(5, 10)):
    X = df[cols].copy().ffill().fillna(0.0)
    feats = pd.DataFrame(index=X.index)

    # basic stats
    feats[f"{name}_mean"] = X.mean(axis=1)
    feats[f"{name}_std"]  = X.std(axis=1)
    feats[f"{name}_skew"] = X.skew(axis=1)
    feats[f"{name}_kurt"] = X.kurtosis(axis=1)

    # PCA
    if X.shape[0] > 1 and X.shape[1] > 1:
        scaled = StandardScaler().fit_transform(X.values)
        pca_n = min(n_pca, min(scaled.shape) - 1)
        if pca_n >= 1:
            comps = PCA(n_components=pca_n, random_state=42).fit_transform(scaled)
            for i in range(pca_n):
                feats[f"{name}_pca{i+1}"] = comps[:, i]

    # rolling on domain mean/std
    for w in roll_windows:
        feats[f"{name}_mean_r{w}"] = feats[f"{name}_mean"].rolling(w, min_periods=1).mean()
        feats[f"{name}_std_r{w}"]  = feats[f"{name}_std"].rolling(w, min_periods=1).mean()

    return feats


def build_domain_features(df, roll_windows=(5, 10)):
    groups = split_domains(df)
    parts = [
        make_domain_features(df, name, cols, n_pca=3, roll_windows=roll_windows)
        for name, cols in groups.items()
    ]
    out = pd.concat(parts, axis=1)

    # cross-domain mean ratios
    dom_means = [c for c in out.columns if c.endswith("_mean")]
    for i in range(len(dom_means)):
        for j in range(i + 1, len(dom_means)):
            a, b = dom_means[i], dom_means[j]
            out[f"{a}_over_{b}"] = out[a] / (out[b].replace(0, np.nan))

    return out


# Block 8 — Build domain+rolling features and add lags (Model 2)

In [ ]:


X_train_dom = build_domain_features(train, roll_windows=(5, 10))
X_test_dom  = build_domain_features(test,  roll_windows=(5, 10))

print("Domain features shapes:", X_train_dom.shape, X_test_dom.shape)
display(X_train_dom.head(2))

def add_lags(df, max_cols=12, lags=(1, 3, 5)):
    """
    Add lag features for a small set of stable columns:
    - all *_mean
    - all *_pca1
    (up to max_cols total to keep things light)
    """
    lag_cols = [c for c in df.columns if c.endswith("_mean")] + \
               [c for c in df.columns if c.endswith("_pca1")]
    lag_cols = lag_cols[:min(max_cols, len(lag_cols))]

    out = df.copy()
    for col in lag_cols:
        for L in lags:
            out[f"{col}_lag{L}"] = df[col].shift(L)
    return out

X_train_adv_lagged = add_lags(X_train_dom, max_cols=12, lags=(1, 3, 5))
X_test_adv_lagged  = add_lags(X_test_dom,  max_cols=12, lags=(1, 3, 5))

Xy_adv = X_train_adv_lagged.join(train_labels[target_cols]).dropna()
y_adv = Xy_adv[target_cols]
X_adv = Xy_adv.drop(columns=target_cols)

X_test_adv = X_test_adv_lagged.fillna(0.0)

print("Advanced (domain+rolling+lags) shapes:")
print("X_adv      :", X_adv.shape)
print("y_adv      :", y_adv.shape)
print("X_test_adv :", X_test_adv.shape)

X_adv_pruned = prune_features(X_adv)
X_test_adv_pruned = prune_features(X_test_adv).reindex(columns=X_adv_pruned.columns, fill_value=0.0)

print("Advanced feature count:", X_adv.shape[1], "→ after prune:", X_adv_pruned.shape[1])


Domain features shapes: (1961, 50) (134, 50)


,US_Stock_mean,US_Stock_std,US_Stock_skew,US_Stock_kurt,US_Stock_pca1,US_Stock_pca2,US_Stock_pca3,US_Stock_mean_r5,US_Stock_std_r5,US_Stock_mean_r10,...,FX_mean_r5,FX_std_r5,FX_mean_r10,FX_std_r10,US_Stock_mean_over_LME_mean,US_Stock_mean_over_JPX_mean,US_Stock_mean_over_FX_mean,LME_mean_over_JPX_mean,LME_mean_over_FX_mean,JPX_mean_over_FX_mean
0,1.232197e+06,4.846176e+06,8.358285,95.072159,-15.41505,-8.548106,-5.942170,1.232197e+06,4.846176e+06,1.232197e+06,...,21.614197,44.198696,21.614197,44.198696,320.290263,NaN,57008.672658,NaN,177.990652,0.0
1,1.240892e+06,4.679417e+06,7.004832,63.595961,-15.36297,-8.575636,-6.017506,1.236545e+06,4.762797e+06,1.236545e+06,...,21.610970,44.178564,21.610970,44.178564,324.819679,NaN,57428.132654,NaN,176.800041,0.0


Advanced (domain+rolling+lags) shapes:
X_adv      : (1133, 86)
y_adv      : (1133, 424)
X_test_adv : (134, 86)
Advanced feature count: 86 → after prune: 86


# Block 9 — TimeSeries CV for Advanced LGBM (Model 2)

In [ ]:


adv_est = LGBMRegressor(
    n_estimators=800,
    learning_rate=0.05,
    num_leaves=64,
    max_depth=-1,
    min_data_in_leaf=20,
    feature_fraction=0.9,
    bagging_fraction=0.8,
    bagging_freq=1,
    random_state=42,
    n_jobs=-1,
    verbosity=-1,
)

adv_model = MultiOutputRegressor(adv_est)
tscv_adv = TimeSeriesSplit(n_splits=5)

adv_metrics = []

print("===== [MODEL 2] Advanced LGBM (domain+rolling+lags) — TimeSeries CV =====")
for fold, (tr_idx, va_idx) in enumerate(tscv_adv.split(X_adv_pruned), 1):
    X_tr, X_va = X_adv_pruned.iloc[tr_idx], X_adv_pruned.iloc[va_idx]
    y_tr, y_va = y_adv.iloc[tr_idx], y_adv.iloc[va_idx]

    adv_model.fit(X_tr, y_tr)
    preds = adv_model.predict(X_va)

    y_true = y_va.to_numpy().ravel()
    y_pred = preds.ravel()

    mse  = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae  = mean_absolute_error(y_true, y_pred)
    r2   = r2_score(y_true, y_pred)

    adv_metrics.append([fold, rmse, mae, r2])
    print(f"[ADVANCED] Fold {fold} → RMSE={rmse:.6f}, MAE={mae:.6f}, R2={r2:.4f}")

adv_df = pd.DataFrame(adv_metrics, columns=["Fold","RMSE","MAE","R2"])
display(adv_df)
print("\nAdvanced means:\n", adv_df.mean(numeric_only=True))


===== [MODEL 2] Advanced LGBM (domain+rolling+lags) — TimeSeries CV =====
[ADVANCED] Fold 1 → RMSE=0.048377, MAE=0.027970, R2=-0.1289
[ADVANCED] Fold 2 → RMSE=0.036918, MAE=0.026005, R2=-0.3997
[ADVANCED] Fold 3 → RMSE=0.041246, MAE=0.029644, R2=-0.3322
[ADVANCED] Fold 4 → RMSE=0.039880, MAE=0.027816, R2=-1.3703
[ADVANCED] Fold 5 → RMSE=0.033983, MAE=0.024038, R2=-0.3167


,Fold,RMSE,MAE,R2
0,1,0.048377,0.027970,-0.128909
1,2,0.036918,0.026005,-0.399698
2,3,0.041246,0.029644,-0.332180
3,4,0.039880,0.027816,-1.370322
4,5,0.033983,0.024038,-0.316653



Advanced means:
 Fold    3.000000
RMSE    0.040081
MAE     0.027094
R2     -0.509552
dtype: float64


# Block 10 — Train advanced model + save submission + compare models

In [ ]:


# Train on full data
adv_model.fit(X_adv_pruned, y_adv)
preds_test_adv = adv_model.predict(X_test_adv_pruned)

sub_adv = pd.DataFrame(preds_test_adv, columns=target_cols)
adv_sub_path = os.path.join(OUTPUT_PATH, "submission_advanced_domain_rolling_lgbm.csv")
sub_adv.to_csv(adv_sub_path, index=False)
print(f"Saved ADVANCED submission to: {adv_sub_path}")

display(sub_adv.head())

# Comparison table
baseline_summary = baseline_df.mean(numeric_only=True).rename("Baseline_LGBM")
adv_summary      = adv_df.mean(numeric_only=True).rename("Advanced_LGBM")

comparison = pd.concat([baseline_summary, adv_summary], axis=1)
print("\n===== Model Comparison (CV Means) =====")
display(comparison)


Saved ADVANCED submission to: ./outputs/submission_advanced_domain_rolling_lgbm.csv


,target_0,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,...,target_414,target_415,target_416,target_417,target_418,target_419,target_420,target_421,target_422,target_423
0,0.015396,-0.009912,0.009221,0.006093,0.004090,0.014419,-0.009844,-0.022022,0.013404,0.012113,...,-0.006800,0.016013,0.019714,-0.009410,0.013058,0.002293,0.007342,-0.012288,-0.012024,0.035299
1,0.004929,-0.008085,0.007588,0.005889,-0.003388,0.009782,-0.009256,-0.017896,-0.002277,0.016275,...,0.006336,0.017886,0.019394,0.025263,0.012049,0.006733,0.005980,-0.008001,-0.009578,0.011268
2,0.001013,-0.003101,0.009924,0.006730,-0.002549,0.009895,-0.009716,-0.017329,-0.001549,0.016438,...,0.004650,0.015875,0.015938,0.023573,0.011836,0.011810,0.003088,-0.006724,-0.006804,0.013877
3,0.010602,-0.008516,-0.001403,-0.000060,0.005737,0.003969,-0.001135,-0.007931,0.008946,0.009283,...,0.004626,0.009835,0.019020,0.009462,0.007707,0.000171,-0.003010,-0.013204,0.014655,0.022536
4,0.008224,-0.005169,-0.009672,-0.001788,0.000982,0.009333,0.002668,-0.005189,0.003515,0.006463,...,0.006273,0.005818,0.024960,0.016062,0.003050,-0.003300,-0.005182,-0.016928,0.009994,0.015151



===== Model Comparison (CV Means) =====


,Baseline_LGBM,Advanced_LGBM
Fold,3.000000,3.000000
RMSE,0.030684,0.040081
MAE,0.022143,0.027094
R2,-0.320688,-0.509552


# Block 11 — Select series for classical forecasting (naive, ARIMA, SARIMA)

In [ ]:


# Pick a few meaningful price series (change if you want)
candidate_series = [
    "LME_CA_Close",
    "LME_ZS_Close",
    "LME_PB_Close",
]

selected_series = [c for c in candidate_series if c in train.columns]

print("Series used for ARIMA/SARIMA/Naive experiments:")
print(selected_series)

# Helper: create train / validation split indices
def make_train_valid_split(s: pd.Series, n_valid: int = 90):
    s = s.dropna()
    if len(s) <= n_valid + 10:
        # if series is short, use 20% for validation
        n_valid = max(10, int(len(s) * 0.2))
    train_s = s.iloc[:-n_valid]
    valid_s = s.iloc[-n_valid:]
    return train_s, valid_s


Series used for ARIMA/SARIMA/Naive experiments:
['LME_CA_Close', 'LME_ZS_Close', 'LME_PB_Close']


# Block 12 — Naive, ARIMA, SARIMA with MAE comparison

In [ ]:


def naive_forecast(train_s: pd.Series, valid_s: pd.Series):
    """
    Naive forecast: y_hat_t = y_(t-1).
    For the first validation point, we use the last train value.
    """
    last_train = train_s.iloc[-1]
    # shift validation series by 1; fill first with last train value
    preds = valid_s.shift(1)
    preds.iloc[0] = last_train
    return preds


def best_arima(train_s: pd.Series, pdq_list=None):
    """
    Simple ARIMA hyperparameter search using AIC.
    """
    if pdq_list is None:
        pdq_list = [(1,0,0), (1,1,0), (1,1,1), (2,1,0), (2,1,1)]

    best_model = None
    best_order = None
    best_aic = np.inf

    for order in pdq_list:
        try:
            m = ARIMA(train_s, order=order).fit(method_kwargs={"warn_convergence": False})
            if m.aic < best_aic:
                best_aic = m.aic
                best_model = m
                best_order = order
        except Exception:
            continue

    return best_model, best_order, best_aic


def best_sarima(train_s: pd.Series, seasonal_period=5, pdq_list=None, seasonal_pdq_list=None):
    """
    Simple SARIMA hyperparameter search using AIC.
    """
    if pdq_list is None:
        pdq_list = [(1,0,0), (1,1,0), (1,1,1)]
    if seasonal_pdq_list is None:
        seasonal_pdq_list = [(0,0,0), (1,0,0), (0,1,1)]

    best_model = None
    best_order = None
    best_seasonal = None
    best_aic = np.inf

    for order in pdq_list:
        for sorder in seasonal_pdq_list:
            try:
                m = SARIMAX(
                    train_s,
                    order=order,
                    seasonal_order=(sorder[0], sorder[1], sorder[2], seasonal_period),
                    enforce_stationarity=False,
                    enforce_invertibility=False,
                ).fit(disp=False)
                if m.aic < best_aic:
                    best_aic = m.aic
                    best_model = m
                    best_order = order
                    best_seasonal = sorder
            except Exception:
                continue

    return best_model, best_order, best_seasonal, best_aic


results = []

for col in selected_series:
    print(f"\n===== Classical forecasting for series: {col} =====")
    s_train, s_valid = make_train_valid_split(train[col])

    # 1) Naive
    naive_pred = naive_forecast(s_train, s_valid)

    # 2) ARIMA (AIC-based search)
    arima_model, arima_order, arima_aic = best_arima(s_train)
    if arima_model is not None:
        arima_pred = arima_model.forecast(len(s_valid))
        arima_pred.index = s_valid.index
    else:
        arima_pred = naive_pred.copy()  # fallback

    # 3) SARIMA (AIC-based search, small grid)
    sarima_model, sarima_order, sarima_seasonal, sarima_aic = best_sarima(s_train)
    if sarima_model is not None:
        sarima_pred = sarima_model.forecast(len(s_valid))
        sarima_pred.index = s_valid.index
    else:
        sarima_pred = naive_pred.copy()  # fallback

    # --- Compute metrics (MAE + RMSE) ---
    y_true = s_valid.values

    for name, y_hat in [
        ("Naive",  naive_pred.values),
        ("ARIMA",  arima_pred.values),
        ("SARIMA", sarima_pred.values),
    ]:
        mae = mean_absolute_error(y_true, y_hat)
        rmse = np.sqrt(mean_squared_error(y_true, y_hat))
        results.append({
            "Series": col,
            "Model":  name,
            "MAE":    mae,
            "RMSE":   rmse,
        })
        print(f"{name:6s} → MAE={mae:.6f}, RMSE={rmse:.6f}")

results_df = pd.DataFrame(results)
print("\n===== Classical Models: MAE / RMSE Comparison =====")
display(results_df)



===== Classical forecasting for series: LME_CA_Close =====


/opt/anaconda3/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, 

Naive  → MAE=75.476444, RMSE=115.755574
ARIMA  → MAE=257.960218, RMSE=356.570278
SARIMA → MAE=292.109826, RMSE=383.660625

===== Classical forecasting for series: LME_ZS_Close =====


/opt/anaconda3/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, 

Naive  → MAE=26.257778, RMSE=32.628758
ARIMA  → MAE=256.801098, RMSE=276.255232
SARIMA → MAE=245.528126, RMSE=265.518527

===== Classical forecasting for series: LME_PB_Close =====


/opt/anaconda3/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, 

Naive  → MAE=14.950889, RMSE=19.315121
ARIMA  → MAE=63.377875, RMSE=77.926355
SARIMA → MAE=74.450316, RMSE=88.920927

===== Classical Models: MAE / RMSE Comparison =====


/opt/anaconda3/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/anaconda3/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


,Series,Model,MAE,RMSE
0,LME_CA_Close,Naive,75.476444,115.755574
1,LME_CA_Close,ARIMA,257.960218,356.570278
2,LME_CA_Close,SARIMA,292.109826,383.660625
3,LME_ZS_Close,Naive,26.257778,32.628758
4,LME_ZS_Close,ARIMA,256.801098,276.255232
5,LME_ZS_Close,SARIMA,245.528126,265.518527
6,LME_PB_Close,Naive,14.950889,19.315121
7,LME_PB_Close,ARIMA,63.377875,77.926355
8,LME_PB_Close,SARIMA,74.450316,88.920927


# Block 13 — Simple hyperparameter search for LGBM (using baseline dataset)

In [ ]:


param_grid = {
    "num_leaves":       [31, 64, 128],
    "learning_rate":    [0.03, 0.05, 0.08],
    "n_estimators":     [400, 800, 1200],
    "min_data_in_leaf": [20, 50],
}

def evaluate_params(params, X, y, n_splits=3):
    """
    TimeSeries CV evaluation for a given set of LGBM params.
    Returns mean RMSE across folds.
    """
    est = LGBMRegressor(
        objective="regression",
        random_state=42,
        n_jobs=-1,
        verbosity=-1,
        **params,
    )
    model = MultiOutputRegressor(est)
    tscv = TimeSeriesSplit(n_splits=n_splits)

    rmses = []
    for tr_idx, va_idx in tscv.split(X):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

        model.fit(X_tr, y_tr)
        preds = model.predict(X_va)

        y_true = y_va.to_numpy().ravel()
        y_pred = preds.ravel()

        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        rmses.append(rmse)

    return float(np.mean(rmses))


search_results = []
print("===== Hyperparameter search for baseline LGBM =====")
for num_leaves in param_grid["num_leaves"]:
    for lr in param_grid["learning_rate"]:
        for n_est in param_grid["n_estimators"]:
            for min_leaf in param_grid["min_data_in_leaf"]:
                params = {
                    "num_leaves": num_leaves,
                    "learning_rate": lr,
                    "n_estimators": n_est,
                    "min_data_in_leaf": min_leaf,
                    "feature_fraction": 0.9,
                    "bagging_fraction": 0.8,
                    "bagging_freq": 1,
                }
                rmse_mean = evaluate_params(params, X_base_pruned, y_base, n_splits=3)
                search_results.append({**params, "RMSE_mean": rmse_mean})
                print(f"params={params} → mean RMSE={rmse_mean:.6f}")

search_df = pd.DataFrame(search_results).sort_values("RMSE_mean")
print("\n===== Hyperparameter Search Results (best first) =====")
display(search_df.head(10))

best_params = search_df.iloc[0].to_dict()
print("\nBest params (from search_df.head(1)):")
print(best_params)


===== Hyperparameter search for baseline LGBM =====
params={'num_leaves': 31, 'learning_rate': 0.03, 'n_estimators': 400, 'min_data_in_leaf': 20, 'feature_fraction': 0.9, 'bagging_fraction': 0.8, 'bagging_freq': 1} → mean RMSE=0.029530
params={'num_leaves': 31, 'learning_rate': 0.03, 'n_estimators': 400, 'min_data_in_leaf': 50, 'feature_fraction': 0.9, 'bagging_fraction': 0.8, 'bagging_freq': 1} → mean RMSE=0.027195
params={'num_leaves': 31, 'learning_rate': 0.03, 'n_estimators': 800, 'min_data_in_leaf': 20, 'feature_fraction': 0.9, 'bagging_fraction': 0.8, 'bagging_freq': 1} → mean RMSE=0.029925
params={'num_leaves': 31, 'learning_rate': 0.03, 'n_estimators': 800, 'min_data_in_leaf': 50, 'feature_fraction': 0.9, 'bagging_fraction': 0.8, 'bagging_freq': 1} → mean RMSE=0.027195
params={'num_leaves': 31, 'learning_rate': 0.03, 'n_estimators': 1200, 'min_data_in_leaf': 20, 'feature_fraction': 0.9, 'bagging_fraction': 0.8, 'bagging_freq': 1} → mean RMSE=0.030067
params={'num_leaves': 31, '

,num_leaves,learning_rate,n_estimators,min_data_in_leaf,feature_fraction,bagging_fraction,bagging_freq,RMSE_mean
53,128,0.08,1200,50,0.9,0.8,1,0.027195
25,64,0.05,400,50,0.9,0.8,1,0.027195
31,64,0.08,400,50,0.9,0.8,1,0.027195
23,64,0.03,1200,50,0.9,0.8,1,0.027195
33,64,0.08,800,50,0.9,0.8,1,0.027195
21,64,0.03,800,50,0.9,0.8,1,0.027195
35,64,0.08,1200,50,0.9,0.8,1,0.027195
19,64,0.03,400,50,0.9,0.8,1,0.027195
17,31,0.08,1200,50,0.9,0.8,1,0.027195
37,128,0.03,400,50,0.9,0.8,1,0.027195



Best params (from search_df.head(1)):
{'num_leaves': 128.0, 'learning_rate': 0.08, 'n_estimators': 1200.0, 'min_data_in_leaf': 50.0, 'feature_fraction': 0.9, 'bagging_fraction': 0.8, 'bagging_freq': 1.0, 'RMSE_mean': 0.02719485450362234}


# Block 14 — Save ALL outputs to CSV files

In [ ]:


OUTPUT_PATH = "./outputs/"
os.makedirs(OUTPUT_PATH, exist_ok=True)

print("==== Saving All Outputs to CSV ====\n")

# 1. Save Baseline LGBM CV Results
if "baseline_df" in globals():
    baseline_cv_path = os.path.join(OUTPUT_PATH, "baseline_cv_metrics.csv")
    baseline_df.to_csv(baseline_cv_path, index=False)
    print(f"Saved Baseline CV metrics → {baseline_cv_path}")
else:
    print("Baseline CV metrics not found (baseline_df missing).")


# 2. Save Advanced LGBM (rolling/domain) CV Results
if "adv_df" in globals():
    advanced_cv_path = os.path.join(OUTPUT_PATH, "advanced_cv_metrics.csv")
    adv_df.to_csv(advanced_cv_path, index=False)
    print(f"Saved Advanced CV metrics → {advanced_cv_path}")
else:
    print("Advanced CV metrics not found (adv_df missing).")


# 3. Save Classical ARIMA/SARIMA/Naive Results
if "results_df" in globals():
    classical_path = os.path.join(OUTPUT_PATH, "classical_forecasting_metrics.csv")
    results_df.to_csv(classical_path, index=False)
    print(f"Saved Classical Forecasting metrics → {classical_path}")
else:
    print("Classical forecasting results not found (results_df missing).")


# 4. Save Hyperparameter Search Results for LGBM
if "search_df" in globals():
    hparam_path = os.path.join(OUTPUT_PATH, "lgbm_hyperparameter_search.csv")
    search_df.to_csv(hparam_path, index=False)
    print(f"Saved Hyperparameter Search results → {hparam_path}")
else:
    print("Hyperparameter search results not found (search_df missing).")


# 5. Save Baseline Final Predictions
if "sub_base" in globals():
    baseline_pred_path = os.path.join(OUTPUT_PATH, "submission_baseline_full_lgbm.csv")
    sub_base.to_csv(baseline_pred_path, index=False)
    print(f"Saved Baseline Final Predictions → {baseline_pred_path}")
else:
    print("Baseline prediction DF missing (sub_base).")


# 6. Save Advanced Final Predictions
if "sub_adv" in globals():
    adv_pred_path = os.path.join(OUTPUT_PATH, "submission_advanced_domain_rolling_lgbm.csv")
    sub_adv.to_csv(adv_pred_path, index=False)
    print(f"Saved Advanced Final Predictions → {adv_pred_path}")
else:
    print("Advanced prediction DF missing (sub_adv).")


print("\n==== All Available Outputs Saved ====")


==== Saving All Outputs to CSV ====

Saved Baseline CV metrics → ./outputs/baseline_cv_metrics.csv
Saved Advanced CV metrics → ./outputs/advanced_cv_metrics.csv
Saved Classical Forecasting metrics → ./outputs/classical_forecasting_metrics.csv
Saved Hyperparameter Search results → ./outputs/lgbm_hyperparameter_search.csv
Saved Baseline Final Predictions → ./outputs/submission_baseline_full_lgbm.csv
Saved Advanced Final Predictions → ./outputs/submission_advanced_domain_rolling_lgbm.csv

==== All Available Outputs Saved ====
